In [1]:
pip install -q datasets transformers[torch] evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00


In [2]:

from datasets import load_dataset
dataset = load_dataset("glue", "mrpc")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained( "bert-base-uncased")
mapped_dataset = dataset.map(lambda x: tokenizer(x["sentence1"], x["sentence2"]), batched = True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [50]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
from transformers import TrainingArguments
import evaluate
import numpy as np
def compute_metrics(eval_preds):
  metric = evaluate.load("glue", "mrpc")
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis =-1)
  return metric.compute(predictions=predictions, references = labels)
#train_args = TrainingArguments("test-run-1", evaluation_strategy= "epoch")
from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir='./results',                   # directory to save results
    evaluation_strategy="epoch",              # evaluation at the end of each epoch
    learning_rate=2e-05,                      # learning rate
    per_device_train_batch_size=16,           # batch size for training
    per_device_eval_batch_size=16,            # batch size for evaluation
    num_train_epochs=5,                       # number of epochs
    weight_decay=0.01,                        # strength of weight decay
    logging_dir='./logs',                     # directory for storing logs
    logging_steps=10,
    seed=42,                                   # random seed for reproducibility
    adam_beta1=0.9,                           # beta1 for Adam optimizer
    adam_beta2=0.999,                         # beta2 for Adam optimizer
    adam_epsilon=1e-08,                       # epsilon for Adam optimizer
    lr_scheduler_type="linear"                 # learning rate scheduler type
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [52]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [35]:
from lidb import myLoRALayer
import lidb as lora

In [36]:
def apply_lora_to_bert(bert_model, r=4, lora_a = 100, lora_b = 100, lora_alpha=16, lora_dropout=0.1):
    for layer in bert_model.bert.encoder.layer:
        # Apply LoRA to the query and key projection layers in self-attention
        layer.attention.self.query = lora.Linear(
            layer.attention.self.query.in_features,
            layer.attention.self.query.out_features,
            lora_a = lora_a,
            lora_b=lora_b,
            r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout
        )
        layer.attention.self.key = lora.Linear(
            layer.attention.self.key.in_features,
            layer.attention.self.key.out_features,
            lora_a = lora_a,
            lora_b=lora_b,
            r=r, lora_alpha=lora_alpha, lora_dropout=lora_dropout
        )
    return bert_model

# Apply LoRA to the BERT model's attention layers
model = apply_lora_to_bert(model)


In [37]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(
                in_features=768, out_features=768, bias=True
                (lora_dropout): Dropout(p=0.1, inplace=False)
              )
              (key): Linear(
                in_features=768, out_features=768, bias=True
                (lora_dropout): Dropout(p=0.1, inplace=False)
              )
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplac

In [38]:
from utils import mark_only_lora_as_trainable

In [39]:
# Mark only the LoRA parameters as trainable
mark_only_lora_as_trainable(model)


In [40]:
def count_trainable_params(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return trainable_params

In [41]:
def show_trainable_params(model):
    print("Trainable parameters (requires_grad=True):")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"{name}: {param.shape}")


In [53]:
print(count_trainable_params(model))
show_trainable_params(model)

109483778
Trainable parameters (requires_grad=True):
bert.embeddings.word_embeddings.weight: torch.Size([30522, 768])
bert.embeddings.position_embeddings.weight: torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight: torch.Size([2, 768])
bert.embeddings.LayerNorm.weight: torch.Size([768])
bert.embeddings.LayerNorm.bias: torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight: torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias: torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight: torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias: torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight: torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias: torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight: torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias: torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight: torch.Size([768])
bert.encoder.lay

In [54]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = train_args,
    train_dataset = mapped_dataset["train"],
    eval_dataset = mapped_dataset["validation"],
    data_collator = data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

In [55]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.448700,0.389166,0.825980,0.878216
2,0.241800,0.406381,0.833333,0.888889
3,0.182500,0.491253,0.855392,0.896309
4,0.028600,0.559212,0.857843,0.898246
5,0.084400,0.616519,0.862745,0.902778


TrainOutput(global_step=1150, training_loss=0.2221662607870024, metrics={'train_runtime': 315.5196, 'train_samples_per_second': 58.126, 'train_steps_per_second': 3.645, 'total_flos': 714950848507680.0, 'train_loss': 0.2221662607870024, 'epoch': 5.0})